In [1]:
! pip3 install --upgrade --user google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 19.5 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import requests
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

## Simple Function Calling

#### Why function calling?
When working with a generative text model, it can be difficult to coerce the LLM to give consistent responses in a structured format such as JSON. Function calling makes it easy to work with LLMs via prompts and unstructured inputs, and have the LLM return a structured response that can be used to call an external function.

You can think of function calling as a way to get structured output from user prompts and function definitions, use that structured output to make an API request to an external system, then return the function response to the LLM to generate a response to the user. In other words, function calling in Gemini extracts structured parameters from unstructured text or messages from users.

Use the Gemini Pro model
The Gemini Pro (gemini-pro) model is designed to handle natural language tasks, multiturn text and code chat, and code generation.

Load the Gemini Pro model:

In [2]:
model = GenerativeModel("gemini-pro")

## Simple function calling

We'll use function calling to set up a weather API request for users to obtain the current conditions in a given location. Function parameters are specified as a Python dictionary in accordance with the OpenAPI JSON schema format.

In [3]:
get_current_weather_func = FunctionDeclaration(
    name="get_current_weather",
    description="Get the current weather in a given location",
    parameters={
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": "Location"
        }
    }
},
)

In [4]:
# Run the following code snippet to define a tool for the LLM to call that includes the get_current_weather_func.

weather_tool = Tool(
    function_declarations=[get_current_weather_func],
)

We wil then instruct the model to generate content. Include the tool that you just created to generate a response:

In [5]:
prompt = "What is the weather like in Boston?"

response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[weather_tool],
)
response

I0000 00:00:1725589967.845362     407 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_current_weather"
        args {
          fields {
            key: "location"
            value {
              string_value: "Boston"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.09521484375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.08251953125
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.1484375
    severity: HARM_SEVERITY_LOW
    severity_score: 0.2060546875
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.140625
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.08154296875
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    prob

In [6]:
response.candidates[0].content.parts[0].function_call

name: "get_current_weather"
args {
  fields {
    key: "location"
    value {
      string_value: "Boston"
    }
  }
}

## Complex Function Calling

Let's generate a function call that has a more complex structure. Let's use the function response to make an API call that converts an address to latitude and longitude coordinates.

Run the following code snippet to define a function declaration within a tool:

In [7]:
get_location = FunctionDeclaration(
    name="get_location",
    description="Get latitude and longitude for a given location",
    parameters={
    "type": "object",
    "properties": {
        "poi": {
            "type": "string",
            "description": "Point of interest"
        },
        "street": {
            "type": "string",
            "description": "Street name"
        },
        "city": {
            "type": "string",
            "description": "City name"
        },
        "county": {
            "type": "string",
            "description": "County name"
        },
        "state": {
            "type": "string",
            "description": "State name"
        },
        "country": {
            "type": "string",
            "description": "Country name"
        },
        "postal_code": {
            "type": "string",
            "description": "Postal code"
        },
    },
},
)
location_tool = Tool(
    function_declarations=[get_location],
)

In [8]:
prompt = """
I want to get the lat/lon coordinates for the following address:
1600 Amphitheatre Pkwy, Mountain View, CA 94043, US
"""
response = model.generate_content(
    prompt,
    generation_config={"temperature": 0},
    tools=[location_tool],
)
response.candidates[0].content.parts[0]

function_call {
  name: "get_location"
  args {
    fields {
      key: "city"
      value {
        string_value: "Mountain View"
      }
    }
    fields {
      key: "country"
      value {
        string_value: "US"
      }
    }
    fields {
      key: "postal_code"
      value {
        string_value: "94043"
      }
    }
    fields {
      key: "state"
      value {
        string_value: "CA"
      }
    }
    fields {
      key: "street"
      value {
        string_value: "1600 Amphitheatre Pkwy"
      }
    }
  }
}

Then, extract the results from the function response and make an API call.

In [9]:
x = response.candidates[0].content.parts[0].function_call.args

url = "https://nominatim.openstreetmap.org/search?"
for i in x:
    url += '{}="{}"&'.format(i, x[i])
url += "format=json"
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (kHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}
x = requests.get(url, headers=headers)
content = x.json()
content

[{'place_id': 377680635,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 2192620021,
  'lat': '37.4217636',
  'lon': '-122.084614',
  'class': 'office',
  'type': 'it',
  'place_rank': 30,
  'importance': 0.6949356759210291,
  'addresstype': 'office',
  'name': 'Google Headquarters',
  'display_name': 'Google Headquarters, 1600, Amphitheatre Parkway, Mountain View, Santa Clara County, California, 94043, United States',
  'boundingbox': ['37.4217136', '37.4218136', '-122.0846640', '-122.0845640']}]

## Function calling in a chat session

Next, let's use the chat model in Gemini to help customers get information about products in a store.

Let's start by defining multiple functions within a tool for getting the product information, the location of stores and placing an order.

In [10]:
get_product_info_func = FunctionDeclaration(
    name="get_product_sku",
    description="Get the SKU for a product",
    parameters={
    "type": "object",
    "properties": {
        "product_name": {
            "type": "string",
            "description": "Product name"
        }
    }
},
)
get_store_location_func = FunctionDeclaration(
    name="get_store_location",
    description="Get the location of the closest store",
    parameters={
    "type": "object",
    "properties": {
        "location": {
            "type": "string",
            "description": "Location"
        }
    }
},
)
place_order_func = FunctionDeclaration(
    name="place_order",
    description="Place an order",
    parameters={
    "type": "object",
    "properties": {
        "product": {
            "type": "string",
            "description": "Product name"
        },
        "account": {
            "type": "integer",
            "description": "Account number"
        },
        "address": {
            "type": "string",
            "description": "Shipping address"
        }
    }
},
)
retail_tool = Tool(
    function_declarations=[get_product_info_func, 
                           get_store_location_func, 
                           place_order_func,
                          ],
)

In [12]:
model = GenerativeModel("gemini-pro", 
                        generation_config={"temperature": 0},
                        tools=[retail_tool])
chat = model.start_chat()

prompt = """
Do you have the Pixel 8 Pro in stock?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_product_sku"
  args {
    fields {
      key: "product_name"
      value {
        string_value: "Pixel 8 Pro"
      }
    }
  }
}

As expected, the response includes a structured function call that we can use to communicate with external systems.

In reality, you would execute function calls against an external system or database. Since this lab focuses on the ability to extract function parameter and generate function calls, you'll use mock data to feed responses back to the model rather that using an actual API server.

Use synthetic data to simulate a response payload from an external API.

In [13]:
api_response = {"sku": "GA04834-US", "in_stock": "yes"}

In [14]:
response = chat.send_message(
    Part.from_function_response(
        name="get_product_sku",
        response={
            "content": api_response,
        }
    ),
)
response.candidates[0].content.parts[0]

text: "Yes, the Pixel 8 Pro is in stock. The SKU is GA04834-US."

In [15]:
prompt = """
Where can I buy one near Mountain View, CA?
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "get_store_location"
  args {
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
  }
}

We get a response with another structured function call, this time it's set up to use a different function from our tool.

Use synthetic data to simulate a response payload from an external API.

In [16]:
api_response = {"store": "1600 Amphitheatre Pkwy, Mountain View, CA 94043, US"}

Again, let's include details from the external API call and generate a response to the user.

In [17]:
response = chat.send_message(
    Part.from_function_response(
        name="get_store_location",
        response={
            "content":  api_response,
        }
    ),
)
response.candidates[0].content.parts[0]

text: "The closest store to Mountain View, CA is located at 1600 Amphitheatre Pkwy, Mountain View, CA 94043, US."

In [18]:
prompt = """
I'd like to order a Pixel 8 Pro and have it shipped to 1155 Borregas Ave, Sunnyvale, CA 94089.
"""

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "place_order"
  args {
    fields {
      key: "account"
      value {
        number_value: 12345.0
      }
    }
    fields {
      key: "address"
      value {
        string_value: "1155 Borregas Ave, Sunnyvale, CA 94089"
      }
    }
    fields {
      key: "product"
      value {
        string_value: "Pixel 8 Pro"
      }
    }
  }
}

Use synthetic data to simulate a response payload from an external API.

In [19]:
api_response = {"payment_status": "paid", "order_number": 12345, "est_arrival": "2 days"}

In [20]:
response = chat.send_message(
    Part.from_function_response(
        name="place_order",
        response={
            "content":  api_response,
        }
    ),
)
response.candidates[0].content.parts[0]

text: "Your order has been placed. Your order number is 12345. Your Pixel 8 Pro will arrive in approximately 2 days. \n\nYour payment has been processed. \n\nYour order will be shipped to 1155 Borregas Ave, Sunnyvale, CA 94089. \n"